# Working with Bechdel Test JSON

In [28]:
from IPython.display import Image
Image(url='https://upload.wikimedia.org/wikipedia/en/b/bf/Dykes_to_Watch_Out_For_%28Bechdel_test_origin%29.jpg')

We retrieved the Bechdel Test data as a JSON file via https://bechdeltest.com/api/v1/doc. This notebook shows the process of importing the JSON file, reorganizing the data, and exporting as a CSV file for ease of use in the analysis. 

The Bechdel Test rating is on a 0-3 scale. 

- 0: Failed
- 1: There's at least two women with names in the movie
- 2: There's at least two women with names in the movie and they talk to each other
- 3: There's at least two women with names in the movie and they talk to each other about something other than a man

In [3]:
import json
import csv

bechdel_movies = open('../Data/bechdel_test_data/bechdel_test_all_movies.json', encoding='utf-8', mode='rt')
bechdel_in = json.loads(bechdel_movies.read())
bechdel_movies.close()


# included the header in advance
bechdel_csv = [['bechdel_id', 'imdb_id', 'rating', 'title', 'year']]
bechdel_convert = []

# with items(), they become tuples and we can use indexing to grab values in the next loop
# need this convert step so you can sort within the list
# iterating over the dict values doesn't retain order
for movie in bechdel_in:
    row_movie = []
    for item in movie.items():
        row_movie.append(item)
    row_movie.sort()
    bechdel_convert.append(row_movie)

# adding tt to match the IMDB ID formatting for tconst    
for movie in bechdel_convert:
    row_movie = []
    for item in movie:
        if item[0] == 'imdbid':
            row_movie.append('tt' + item[1])
        else:
            row_movie.append(item[1])
    bechdel_csv.append(row_movie)

#print(bechdel_csv[0:200])

bechdel_write = open('bechdel.csv', encoding='utf-8', mode='wt', newline='')
bechdel_out = csv.writer(bechdel_write)
bechdel_out.writerows(bechdel_csv)

bechdel_write.close()

# Checking for Errors

In [22]:
import pandas as pd

bechdel_df = pd.DataFrame(bechdel_csv)
bechdel_df.head()

,0,1,2,3,4
0,bechdel_id,imdb_id,rating,title,year
1,8040,tt0392728,0,Roundhay Garden Scene,1888
2,5433,tt0000003,0,Pauvre Pierrot,1892
3,5444,tt0000014,0,Tables Turned on the Gardener,1895
4,6200,tt0132134,0,"Execution of Mary, Queen of Scots, The",1895


In [23]:
bechdel_df.columns = bechdel_df.iloc[0]

bechdel_df.drop(index=0, inplace=True)

bechdel_df.head()

,bechdel_id,imdb_id,rating,title,year
1,8040,tt0392728,0,Roundhay Garden Scene,1888
2,5433,tt0000003,0,Pauvre Pierrot,1892
3,5444,tt0000014,0,Tables Turned on the Gardener,1895
4,6200,tt0132134,0,"Execution of Mary, Queen of Scots, The",1895
5,4982,tt0000091,0,"House of the Devil, The",1896


In [24]:
bechdel_df.shape

(7721, 5)

First, we'll see if there are any duplicates via imdb_id. By using groupby and count, we can see if there are multiple values.

In [25]:
temp_bechdel1 = bechdel_df.groupby('imdb_id')['title'].count()
temp_bechdel1.sort_values(ascending=False)

imdb_id
tt2043900    2
tt2457282    2
tt0086425    2
tt0035279    2
tt0117056    2
tt2083355    2
tt7875464    1
tt0118015    1
tt0117998    1
tt0117991    1
tt0117968    1
tt0117979    1
tt0117951    1
tt0117887    1
tt0117876    1
tt0117802    1
tt0117797    1
tt0118105    1
tt0118113    1
tt0118111    1
tt0117737    1
tt0118414    1
tt0118550    1
tt0118570    1
tt0118571    1
tt0118583    1
tt0118589    1
tt0118604    1
tt0118617    1
tt0118636    1
            ..
tt1196141    1
tt1185416    1
tt1189340    1
tt1185616    1
tt1185836    1
tt1186367    1
tt1186369    1
tt1186373    1
tt1186830    1
tt1187043    1
tt1187044    1
tt1187064    1
tt1188113    1
tt1188729    1
tt1189073    1
tt1190072    1
tt1195478    1
tt1190080    1
tt1190536    1
tt1190539    1
tt1191111    1
tt1192628    1
tt1193138    1
tt1194173    1
tt1194238    1
tt1194263    1
tt1194417    1
tt1194424    1
tt1194577    1
tt0000003    1
Name: title, Length: 7715, dtype: int64

This uncovered 6 duplicates, which will have to be checked and removed. We did this in our data filtering and cleaning process in SQL with the IMDB title.principals table. Ultimately, we exported an unduplicated copy (bechdel_test_updated.csv) of the Bechdel Test data from this process and continued to work off that. 

Let's also check for duplicates in the bechdel_id to see if there's any errors there.

In [27]:
temp_bechdel2 = bechdel_df.groupby('bechdel_id')['title'].count()
temp_bechdel2.sort_values(ascending=False)

bechdel_id
996     1
34      1
3428    1
3429    1
343     1
3430    1
3431    1
3432    1
3433    1
3434    1
3435    1
3436    1
3437    1
3438    1
3439    1
344     1
3440    1
3441    1
3442    1
3443    1
3444    1
3445    1
3446    1
3447    1
3448    1
3449    1
345     1
3450    1
3451    1
3427    1
       ..
5920    1
5895    1
5907    1
5896    1
5897    1
5898    1
5899    1
59      1
590     1
5900    1
5901    1
5902    1
5903    1
5904    1
5906    1
5908    1
592     1
5909    1
591     1
5910    1
5911    1
5912    1
5913    1
5914    1
5915    1
5916    1
5917    1
5918    1
5919    1
1       1
Name: title, Length: 7721, dtype: int64

There are no duplicates in the bechdel_id column. We won't check for duplicates in the title column because many movies have the same name or they're remakes. 